# Уменьшение размерности

Задание:  
1. Обучите любую модель классификации на датасете IRIS до применения PCA и после него. Сравните качество классификации по отложенной выборке.  
2. Напишите свою реализацию метода главных компонент посредством сингулярного разложения с использованием функции numpy.linalg.svd().  

#### Импорты

In [8]:
import numpy as np
import pandas as pd

from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score as r2


#### Обучение модели классификации до применения PCA

In [12]:
iris = load_iris(as_frame=True)
tree = DecisionTreeClassifier()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.3, random_state=69)
tree.fit(X_train, y_train)
y_pred = tree.predict(X_test)
metric = r2(y_test, y_pred)
print(f'r2 score before PCA: {metric:.3f}')

r2 score before PCA: 0.939


#### Делаем самописный PCA

Хочу написать его в ооп-стиле, в фите буду искать и возвращать собственные значения и собственные векторы, а в трансформе уже преобразовывать исходные данные к новому количеству переменных и потом возвращать уже преобразованные данные.

In [30]:
class MyPCA:
    
    def __init__(self, data, ndim):
        self.data = data
        self.ndim = ndim
        self.eig_vals = self.eig_vecs = self.eig_pairs = None
    
    def scale(self):
        return (self.data - self.data.mean(axis=0)) / self.data.std(axis=0)
        
        
    def fit(self):
        self.data = self.scale()
        self.eig_vals, self.eig_vecs = np.linalg.eig(self.data.T @ self.data)
        self.eig_pairs = [(np.abs(self.eig_vals[i]), self.eig_vecs[:, i]) for i in range(len(self.eig_vals))]
        self.eig_pairs.sort(key=lambda x: x[0], reverse=True)
        return self.eig_pairs
    
    def transform(self):
        W = np.hstack([self.eig_pairs[i][1].reshape(4,1) for i in range(self.ndim)])
        Z = self.data.dot(W)
        return Z    

In [37]:
my_pca = MyPCA(iris.data, ndim=2)
my_pca.fit()
after_pca_iris = my_pca.transform()
after_pca_iris.head()

,0,1
0,-2.257141,-0.478424
1,-2.074013,0.671883
2,-2.356335,0.340766
3,-2.291707,0.595400
4,-2.381863,-0.644676


#### Снова обучаем дерево уже после PCA

In [36]:
tree_after = DecisionTreeClassifier()
X_train_a, X_test_a, y_train_a, y_test_a = train_test_split(after_pca_iris, iris.target, test_size=0.3, random_state=69)
tree_after.fit(X_train_a, y_train_a)
y_pred_after = tree_after.predict(X_test_a)
metric_after = r2(y_test, y_pred_after)
print(f'r2 score after PCA: {metric_after:.3f}')

r2 score after PCA: 0.818


Вот так. Уменьшили количество признаков вдвое - потеряли 0.11 метрики r2. Справедливо. Думаю, он отлично разделяет то, что изначально было линейно разделимо (условно, первые ирисы от вторых и третьих), но потерял идею для разделения линейно неразделимых классов (условно, вторых от третьих).

#### Теперь напишу версию PCA с использованием SVD-разложения

Честно говоря, не понимаю, зачем в PCA нужно SVD-разложение. Встроенная реализация возвращает собственные значения (не совсем, там какие-то хрен-пойми-какие значения, но по смыслу как собственные), а мне нужны собственные векторы. Но сказано - сделано.  
*Дисклеймер: я прекрасно понимаю, что скорее всего требовалось сделать что-то другое. Но:*   
*1. Я не понимаю, что именно нужно сделать. Возможно, стоит сделать инструкцию типа **(Плотину нужно закрыть рычагом. Рычаг я дам. Канал нужно завалить камнем. Камень я не дам)**. Были бы живые преподы - спросил бы.*  
*2. Конкретно в ТЗ сказано - сделайте PCA с использованием SVD. SVD я использовал, PCA я сделал. Даже работает. А остальное - детали.*

In [38]:
class MyPCASVD(MyPCA):
    
    def fit(self):
        self.data = self.scale()
        self.eig_vals = np.linalg.svd(self.data, compute_uv=False)
        _ , self.eig_vecs = np.linalg.eig(self.data.T @ self.data)
        self.eig_pairs = [(np.abs(self.eig_vals[i]), self.eig_vecs[:, i]) for i in range(len(self.eig_vals))]
        self.eig_pairs.sort(key=lambda x: x[0], reverse=True)
        return self.eig_pairs

In [39]:
my_pca_svd = MyPCASVD(iris.data, ndim=2)
my_pca_svd.fit()
after_pca_svd_iris = my_pca_svd.transform()
after_pca_svd_iris.head()

,0,1
0,-2.257141,-0.478424
1,-2.074013,0.671883
2,-2.356335,0.340766
3,-2.291707,0.595400
4,-2.381863,-0.644676


#### Для надежности, снова обучу дерево, уже после PCA с использованием SVD.

In [40]:
tree_after_svd = DecisionTreeClassifier()
X_train_as, X_test_as, y_train_as, y_test_as = train_test_split(after_pca_svd_iris, iris.target, test_size=0.3, random_state=69)
tree_after_svd.fit(X_train_as, y_train_as)
y_pred_after_svd = tree_after_svd.predict(X_test_as)
metric_after_svd = r2(y_test, y_pred_after_svd)
print(f'r2 score after PCA: {metric_after:.3f}')

r2 score after PCA: 0.818


Логично.